In [89]:
import torch
import yaml
from utils.builder import build_vae_model
from models.condition import TextEncoder 
from models.mar import MAREncoderDecoder
from thsolver.config import parse_args
import sys 
import ocnn
import os
import copy
from ognn.octreed import OctreeD
from utils import utils
torch.cuda.set_device(2)

In [90]:
log_path = '/mnt/sdc/wangrh/workspace/OctAR-solver/logs/objaverse/mar_bv32_b24_p1024_d8_text'

sys.argv = ['']  # Reset sys.argv
sys.argv.extend(['--config', log_path + '/all_configs.yaml'])
flags = parse_args(backup=False)
flags

CfgNode({'BASE': ['configs/shapenet_octar.yaml'], 'SOLVER': CfgNode({'alias': '', 'gpu': (0, 1, 2, 3, 4, 5, 6, 7), 'run': 'train', 'logdir': 'logs/objaverse/mar_bv32_b24_p1024_d8_text', 'ckpt': '', 'ckpt_num': 10, 'type': 'adamw', 'weight_decay': 0.01, 'clip_grad': -1.0, 'max_epoch': 400, 'warmup_epoch': 20, 'warmup_init': 0.001, 'eval_epoch': 1, 'eval_step': -1, 'test_every_epoch': 1, 'log_per_iter': 50, 'best_val': 'min:loss', 'zero_grad_to_none': False, 'use_amp': True, 'lr_type': 'constant', 'lr': 1e-05, 'lr_min': 0.0001, 'gamma': 0.1, 'milestones': (120, 180), 'lr_power': 0.9, 'port': 10001, 'progress_bar': True, 'rand_seed': 0, 'empty_cache': 50, 'expand_ckpt': False, 'step_size': (160, 240), 'resolution': 256, 'save_sdf': False, 'sdf_scale': 0.9}), 'DATA': CfgNode({'train': CfgNode({'name': 'shapenet_vae', 'disable': False, 'pin_memory': True, 'depth': 8, 'full_depth': 3, 'orient_normal': '', 'distort': False, 'scale': 0.0, 'uniform': False, 'jitter': 0.0, 'interval': (1, 1, 1),

In [91]:
device = 'cuda'

model = MAREncoderDecoder(vqvae_config=flags.MODEL.VQVAE, **flags.MODEL.GPT)
vqvae = build_vae_model(flags.MODEL.VQVAE)
text_encoder = TextEncoder(flags.MODEL.GPT.condition_encoder)

Some weights of the model checkpoint at /mnt/sdc/wangrh/workspace/pretrained-model/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.19.mlp.fc2.bias', 'vision_model.encoder.layers.4.mlp.fc1.weight', 'vision_model.encoder.layers.0.self_attn.out_proj.weight', 'vision_model.encoder.layers.19.self_attn.out_proj.bias', 'vision_model.encoder.layers.9.self_attn.v_proj.weight', 'vision_model.encoder.layers.9.self_attn.k_proj.weight', 'vision_model.encoder.layers.10.self_attn.out_proj.weight', 'vision_model.encoder.layers.10.self_attn.q_proj.weight', 'vision_model.encoder.layers.12.self_attn.q_proj.weight', 'vision_model.encoder.layers.5.layer_norm2.bias', 'vision_model.encoder.layers.19.self_attn.q_proj.weight', 'vision_model.encoder.layers.23.mlp.fc2.weight', 'vision_model.encoder.layers.20.mlp.fc2.bias', 'vision_model.encoder.layers.5.layer_norm1.bias', 'vision_model.encoder.layers.3.mlp.fc1.bias', 'vision_model.encoder.layers.16.self_attn.v_

In [92]:
vqvae_checkpoint = torch.load(flags.MODEL.vqvae_ckpt, weights_only=True, map_location="cpu")
vqvae.load_state_dict(vqvae_checkpoint)
print("Load VQVAE from", flags.MODEL.vqvae_ckpt)

Load VQVAE from saved_ckpt/vqvae_objv_huge_bsq32.pth


In [93]:
ar_checkpoint = log_path + '/checkpoints/00070.model.pth'
model_checkpoint = torch.load(ar_checkpoint, map_location="cpu")
model.load_state_dict(model_checkpoint)
print("Load MAR from", ar_checkpoint)

Load MAR from /mnt/sdc/wangrh/workspace/OctAR-solver/logs/objaverse/mar_bv32_b24_p1024_d8_text/checkpoints/00070.model.pth


In [94]:
model = model.to(device)
vqvae = vqvae.to(device)
text_encoder = text_encoder.to(device)

In [95]:
text = 'A scary monster with a big mouth and sharp teeth.'

In [96]:
depth = flags.DATA.test.depth
full_depth = flags.DATA.test.full_depth
depth_stop = flags.MODEL.depth_stop

with torch.no_grad():
    condition = text_encoder(text, device=device)
    octree_out = ocnn.octree.init_octree(
        depth=depth,
        full_depth=full_depth,
        batch_size=1,
        device=device,
    )
    with torch.autocast('cuda', enabled=flags.SOLVER.use_amp):
        octree_out, vq_code = model.generate(
            octree=octree_out,
            depth_low=full_depth,
            depth_high=depth_stop,
            vqvae=vqvae,
            condition=condition,
            cfg_scale=None
        )

  0%|          | 0/64 [00:00<?, ?it/s]

100%|██████████| 256/256 [00:55<00:00,  4.59it/s]


In [97]:
print(condition.shape)
export_path = f'results-inference/{text}'

index = 'output'

# Export octrees
for d in range(full_depth+1, depth_stop+1):
    utils.export_octree(octree_out, d, os.path.join(
        log_path ,export_path), index=f'octree_{d}')

# Decode the mesh
for d in range(depth_stop, depth):
    split_zero_d = torch.zeros(
          octree_out.nnum[d], device=octree_out.device).long()
    octree_out.octree_split(split_zero_d, d)
    octree_out.octree_grow(d + 1)
doctree_out = OctreeD(octree_out)
with torch.no_grad():
    output = vqvae.decode_code(
        vq_code, depth_stop, doctree_out,
        copy.deepcopy(doctree_out), update_octree=True)

# extract the mesh
utils.create_mesh(
    output['neural_mpu'],
    os.path.join(log_path, export_path, f"output.obj"),
    size=flags.SOLVER.resolution,
    level=0.002, clean=True,
    bbmin=-flags.SOLVER.sdf_scale,
    bbmax=flags.SOLVER.sdf_scale,
    mesh_scale=flags.DATA.test.points_scale,
    save_sdf=flags.SOLVER.save_sdf)

# Save the text:
if text is not None:
    os.makedirs(os.path.join(log_path, export_path, "text"), exist_ok=True)
    with open(os.path.join(log_path, export_path, f"input.txt"), "w") as f:
        f.write(text + '\n')

torch.Size([1, 13, 768])


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  6.77it/s]


: 